## Pre-process the CT images

In [1]:
import os
import glob
import SimpleITK as sitk
from scipy import ndimage
import numpy as np

In [2]:
val_patient=[2, 7, 13, 21, 27, 33, 34, 41]
test_patient=[9, 12, 16, 22, 25, 36, 38, 40]

In [3]:
def resample_mha(path, new_spacing, interpolator=sitk.sitkLinear, default_value=-1000):
    """
    Given a path to an sitk image (e.g. .mha file) it resamples the image to a given spacing
    :param path: path to image
    :param new_spacing: spacing to which image is resampled, e.g. [1,1,1] for isotropic resampling
    :param interpolator: which interpolator to use, e.g. sitk.sitkLinear, sitk.sitkNearestNeighbor, sitk.sitkBSpline
    :param default_value: pixel value when a transformed pixel is outside of the image. The default default pixel value is 0.
    :return: resampled image executable
    """
    # load sitk image and create instance of ResampleImageFilter class 
    image = sitk.ReadImage(path)
    resample = sitk.ResampleImageFilter()
    
    # set parameters for resample instance
    resample.SetInterpolator(interpolator)
    resample.SetDefaultPixelValue(default_value)
    resample.SetOutputDirection(image.GetDirection())
    resample.SetOutputOrigin(image.GetOrigin())
#    print('Original origin: ' + str(image.GetOrigin()))
#    print('New origin: ' + str(resample.GetOutputOrigin()))
    resample.SetOutputSpacing(new_spacing)
    
    # compute new size and set parameters for resample instance
    orig_size = np.array(image.GetSize(), dtype=np.int)
    orig_spacing = np.array(image.GetSpacing(), dtype=np.float)
    new_spacing = np.array(new_spacing, dtype=np.float)
    new_size = orig_size*(orig_spacing/new_spacing)
    #new_size = orig_size
    new_size = np.ceil(new_size).astype(np.int) #  image dimensions are in integers
    new_size = [int(s) for s in new_size]

    resample.SetSize(new_size) 
    #print('Original spacing: ' + str(orig_spacing)) # [1.14453125 1.14453125 3.  ]
    #print('New spacing: ' + str(new_spacing)) # [1. 1. 1.]
    #print('Original size: ' + str(orig_size)) # [512 512 135]
    #print('New size: ' + str(new_size)) # [586, 586, 405]

   # perform the resampling with the params set above
    new_image = resample.Execute(image)
    
    return new_image

## Convert CT and CBCT to PNG

In [1]:
# Convert CT to PNG
import os
import glob
import imageio
from PIL import Image
from skimage.morphology import square
DATA_NAME = 'train'
IMAGE_TYPE = '.png'
patients_folders='/home/(ACCOUNT_NAME)/monai_test/Data/Data/'

Data_root_2d='/home/(ACCOUNT_NAME)/monai_test/Unpaired_MR_to_CT_Image_Synthesis-master/data/datasets/MR2CT/'


target_path_ct_train = Data_root_2d + DATA_NAME + 'B'
target_path_ct_test = Data_root_2d + DATA_NAME + 'B'
target_path_ct_val = Data_root_2d + DATA_NAME + 'B'
target_path_ct_mask_train = Data_root_2d + 'mask' + 'B'
target_path_ct_mask_test = Data_root_2d + 'mask' + 'B_test'
target_path_ct_mask_val = Data_root_2d + 'mask' + 'B_val'

new_spacing=[1,1,1]

for f in range(len(os.listdir(patients_folders))):
    patient_folder = patients_folders + os.listdir(patients_folders)[f]
    patient_folders=glob.glob(os.path.join(patient_folder, "*pm_out"))
    for ff in range(len(os.listdir(patient_folders[0]))):
        path_dcm = patient_folders[0] + '/' + os.listdir(patient_folders[0])[ff]
        if "image" in path_dcm:
            # resample the CT images with new spacing [1,1,1]
            image_ct=resample_mha(path_dcm,new_spacing, interpolator=sitk.sitkLinear)
            image_array_ct = sitk.GetArrayFromImage(image_ct) 
            image_array_ct = image_array_ct[:,19:531 , 19:531] #crop to original size
            #print(image_array_ct.shape)
            image_array_ct = (image_array_ct + 1024.0)/ 65536.0  
            image_array_ct[image_array_ct<0]=0
            current_image_ct=image_array_ct[0,:,:]
            
            #remove the couch
            sy=ndimage.sobel(current_image_ct,axis=0,mode='constant')
            sum_sy=np.sum(sy, axis = 1)
            peak_sy=max(sum_sy)
            which=sum_sy==peak_sy
            table_y=np.where(which==True)
            #print(table_y[0])
            current_image_ct[int(table_y[0]):-1,:]=0
            
            
            # mask the trainB by mask_without_holes 
            from skimage import morphology
            #mask = current_image_ct>0.007
            mask_1 = current_image_ct>0.007
            mask = current_image_ct>0.009
            mask_with_hole = morphology.remove_small_objects(mask, min_size=100) #re-train
            mask_without_hole = morphology.remove_small_holes(mask_with_hole, area_threshold=10000)
            #do erosion 
            mask_erosion = morphology.erosion(mask_without_hole,square(20))
            current_image_ct[mask_without_hole<1]=0
            
            # save CT images (trainB)
            if current_image_ct.max()>0.05:
                print(current_image_ct.max())
                print('CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE)
                print(path_dcm)
  
            ct_intensity_min = 0.00
            ct_intensity_max = 0.05
            current_image_ct = (current_image_ct - ct_intensity_min) / (ct_intensity_max - ct_intensity_min) * 65535
            current_image_ct = np.asarray(current_image_ct, dtype='uint16') 
            if np.sum(current_image_ct) != 0:
                if f in val_patient:
                    png_output_ct=str(os.path.join(target_path_ct_val +'/'+'CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE))
                    #imageio.imwrite(png_output_ct, current_image_ct) 
                elif f in test_patient:
                    png_output_ct=str(os.path.join(target_path_ct_test +'/'+'CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE))
                    #imageio.imwrite(png_output_ct, current_image_ct) 
                else:
                    png_output_ct=str(os.path.join(target_path_ct_train +'/'+'CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE))
                    #imageio.imwrite(png_output_ct, current_image_ct) 
            
            
            # save CT body outline (maskB)
            mask_intensity_min = 0.00
            mask_intensity_max = 1.00
            mask_erosion = (mask_erosion - mask_intensity_min) / (mask_intensity_max - mask_intensity_min) * 65535
            mask_erosion = np.asarray(mask_erosion, dtype='uint16') 
            if np.sum(mask_without_hole) != 0:
                if f in val_patient:
                    png_output_ct_mask=str(os.path.join(target_path_ct_mask_val +'/'+'CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE))
                    #imageio.imwrite(png_output_ct_mask, mask_without_hole) 
                elif f in test_patient:
                    png_output_ct_mask=str(os.path.join(target_path_ct_mask_test +'/'+'CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE))
                    #imageio.imwrite(png_output_ct_mask, mask_without_hole) 
                else:
                    png_output_ct_mask=str(os.path.join(target_path_ct_mask_train +'/'+'CT00'+ str(f)+ '_'+ str(ff)+IMAGE_TYPE))
                    #imageio.imwrite(png_output_ct_mask, mask_without_hole) 

FileNotFoundError: [Errno 2] No such file or directory: '/home/(ACCOUNT_NAME)/monai_test/Data/Data/'

## Convert CBCT to PNG

In [ ]:
## Convert original CBCT to PNG
import os
import glob
import imageio
DATA_NAME = 'train'
IMAGE_TYPE = '.png'
patients_folders='/home/(ACCOUNT_NAME)/monai_test/Data/Data/'

Data_root_2d='/home/(ACCOUNT_NAME)/monai_test/Unpaired_MR_to_CT_Image_Synthesis-master/data/datasets/MR2CT/'

target_path_cbct_train = Data_root_2d + DATA_NAME + 'A'
target_path_cbct_test = Data_root_2d + DATA_NAME + 'A'
target_path_cbct_val = Data_root_2d + DATA_NAME + 'A'
target_path_cbct_mask_train = Data_root_2d + 'mask' + 'A'
target_path_cbct_mask_test = Data_root_2d + 'mask' + 'A_test'
target_path_cbct_mask_val = Data_root_2d + 'mask' + 'A_val'

if not os.path.exists(target_path_cbct):
    os.makedirs(target_path_cbct)
    
#read CBCT to png
reader_cbct = sitk.ImageFileReader()

patients_folders='/home/(ACCOUNT_NAME)/monai_test/Data/Data/'

for f in range(len(os.listdir(patients_folders))):
    patient_folder = patients_folders + os.listdir(patients_folders)[f]
    patient_folders=glob.glob(os.path.join(patient_folder, "CBCT*", "CBCT_HU_orig*"))
    #print(patient_folders)
    for ff in range(len(os.listdir(patient_folders[0]))):
        path_dcm = patient_folders[0] + '/' + os.listdir(patient_folders[0])[ff]
        if "image" in path_dcm:
            reader_cbct.SetFileName(path_dcm)
            image_cbct = reader_cbct.Execute()
            image_array_cbct = sitk.GetArrayFromImage(image_cbct) 

            if image_cbct.GetOrigin()[2]<106 and image_cbct.GetOrigin()[2]>-101:
                image_array_cbct = (image_array_cbct + 1024.0)/ 65536.0  
                
                mask=image_array_cbct>0.008  #change it to 0.008 #pading
            
                struct=np.ones((1,10,10))
                mask=ndimage.binary_erosion(mask,structure=struct).astype(mask.dtype)
                mask=ndimage.binary_dilation(mask,structure=struct).astype(mask.dtype)
                
                mask_with_hole = morphology.remove_small_objects(mask, min_size=2500) 
                current_image_cbct_mask = morphology.remove_small_holes(mask_with_hole, area_threshold=10000) 
                
                image_array_cbct[current_image_cbct_mask<1]=0
                
                #padding to [512,512]
                image_array_cbct = np.pad(image_array_cbct,[(0,0),(51,51),(51,51)],mode="edge")
                current_image_cbct=image_array_cbct[0,:,:]

                #norm to 255 for png
                if current_image_cbct.max()>0.04:
                    print(current_image_cbct.max())
                    print('CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE)
                    print(path_dcm)
                    #save these 
                    
                cbct_intensity_min = 0.00
                cbct_intensity_max = 0.04
                current_image_cbct = (current_image_cbct - cbct_intensity_min) / (cbct_intensity_max - cbct_intensity_min) * 65535
                current_image_cbct = np.asarray(current_image_cbct, dtype='uint16') 
                if np.sum(current_image_cbct) != 0:
                    if f in val_patient:
                        png_output_cbct=str(os.path.join(target_path_cbct_val +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        #imageio.imwrite(png_output_cbct, current_image_cbct)  
                    elif f in test_patient:
                        png_output_cbct=str(os.path.join(target_path_cbct_test +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        #imageio.imwrite(png_output_cbct, current_image_cbct) 
                    else:
                        png_output_cbct=str(os.path.join(target_path_cbct_train +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        #imageio.imwrite(png_output_cbct, current_image_cbct) 
                
                #MASK A
                current_image_cbct_mask=image_array_cbct[0,:,:]>0.004
            
                #remove holes in CBCT masks
                mask_with_hole = morphology.remove_small_objects(current_image_cbct_mask, min_size=2500)
                #mask_with_hole = morphology.remove_small_objects(current_image_cbct_mask, min_size=100)
                current_image_cbct_mask = morphology.remove_small_holes(mask_with_hole, area_threshold=10000) 
            

                #norm to 255 for png
                cbct_intensity_min = 0.00
                cbct_intensity_max = 1
                current_image_cbct_mask = (current_image_cbct_mask - cbct_intensity_min) / (cbct_intensity_max - cbct_intensity_min) * 65535
                current_image_cbct_mask = np.asarray(current_image_cbct_mask, dtype='uint16') 
                if np.sum(current_image_cbct_mask) != 0:
                    if f in val_patient:
                        png_output_cbct_mask=str(os.path.join(target_path_cbct_mask_val +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        #imageio.imwrite(png_output_cbct_mask, current_image_cbct_mask)  
                    elif f in test_patient:
                        png_output_cbct_mask=str(os.path.join(target_path_cbct_mask_test +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        #imageio.imwrite(png_output_cbct_mask, current_image_cbct_mask) 
                    else:
                        png_output_cbct_mask=str(os.path.join(target_path_cbct_mask_train +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        #imageio.imwrite(png_output_cbct_mask, current_image_cbct_mask) 
            

In [ ]:
## Convert corrected CBCT to PNG
import os
import glob
import imageio
from skimage import morphology
DATA_NAME = 'train'
IMAGE_TYPE = '.png'
patients_folders='/home/(ACCOUNT_NAME)/monai_test/Data/Data/'
Data_root_2d='/home/(ACCOUNT_NAME)/monai_test/Unpaired_MR_to_CT_Image_Synthesis-master/data/datasets/MR2CT/'

target_path_cbct_train = Data_root_2d + DATA_NAME + 'A_cor'
target_path_cbct_test = Data_root_2d + DATA_NAME + 'A_cor'
target_path_cbct_val = Data_root_2d + DATA_NAME + 'A_cor'

if not os.path.exists(target_path_cbct):
    os.makedirs(target_path_cbct)

#read CBCT to png
reader_cbct = sitk.ImageFileReader()

patients_folders='/home/(ACCOUNT_NAME)/monai_test/Data/Data/'


for f in range(len(os.listdir(patients_folders))):
    patient_folder = patients_folders + os.listdir(patients_folders)[f]
    patient_folders=glob.glob(os.path.join(patient_folder, "CBCT*", "CBCT_HU_cor*"))
    for ff in range(len(os.listdir(patient_folders[0]))):
    #for ff in range(10,79):
        path_dcm = patient_folders[0] + '/' + os.listdir(patient_folders[0])[ff]
        if "image" in path_dcm:
            reader_cbct.SetFileName(path_dcm)
            image_cbct = reader_cbct.Execute()
            image_array_cbct = sitk.GetArrayFromImage(image_cbct) 
            
            if image_cbct.GetOrigin()[2]<106 and image_cbct.GetOrigin()[2]>-101:
                image_array_cbct = (image_array_cbct + 1024.0)/ 65536.0  
                mask=image_array_cbct>0.008  #change it to 0.008 #pading
            
                struct=np.ones((1,10,10))
                mask=ndimage.binary_erosion(mask,structure=struct).astype(mask.dtype)
                mask=ndimage.binary_dilation(mask,structure=struct).astype(mask.dtype)
                
                mask_with_hole = morphology.remove_small_objects(mask, min_size=2500) 
                current_image_cbct_mask = morphology.remove_small_holes(mask_with_hole, area_threshold=10000) 
                image_array_cbct[current_image_cbct_mask<1]=0
                
                #padding to [512,512]
                image_array_cbct = np.pad(image_array_cbct,[(0,0),(51,51),(51,51)],mode="edge")
            
                current_image_cbct=image_array_cbct[0,:,:]
                
                if current_image_cbct.max()>0.04:
                    print(current_image_cbct.max())
                    print('CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE)
                    print(path_dcm)
                    #save these 
            
                #norm to 255 for png
                #print(current_image_cbct.max())
                cbct_intensity_min = 0.00
                cbct_intensity_max = 0.06
                current_image_cbct = (current_image_cbct - cbct_intensity_min) / (cbct_intensity_max - cbct_intensity_min) * 65535
                current_image_cbct = np.asarray(current_image_cbct, dtype='uint16') 
                
                if np.sum(current_image_cbct) != 0:
                    imageio.imwrite(png_output_cbct, current_image_cbct)           
                #print(png_output_cbct)
                
                if np.sum(current_image_cbct) != 0:
                    if f in val_patient:
                        png_output_cbct=str(os.path.join(target_path_cbct_val +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        imageio.imwrite(png_output_cbct, current_image_cbct)  
                    elif f in test_patient:
                        png_output_cbct=str(os.path.join(target_path_cbct_test +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        imageio.imwrite(png_output_cbct, current_image_cbct) 
                    else:
                        png_output_cbct=str(os.path.join(target_path_cbct_train +'/'+'CBCT00'+ str(f)+ '_'+ str(image_cbct.GetOrigin()[2])+IMAGE_TYPE))
                        imageio.imwrite(png_output_cbct, current_image_cbct) 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(1, figsize=(20,5))   
plt.subplot(1,2,1)
plt.imshow(current_image_cbct[:,:], cmap="gray") 
plt.colorbar()
plt.subplot(1,2,2)
plt.imshow(image_array_cbct[0,:,:], cmap="gray") 
plt.colorbar()